In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir('gdrive/My Drive/text_mining/Projet')

In [0]:
!ls

'BBC News Sample Solution.csv'	'BBC News Train.csv'
'BBC News Test.csv'		 my-submission.csv


In [0]:
from google.colab import files
# Install Kaggle library

uploaded = files.upload()

#classification

In [0]:

import numpy as np
import pandas as pd

In [0]:
news = pd.read_csv ('BBC News Train.csv')



In [0]:
news.head()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business


In [0]:
news=news[['Category','Text']]

In [0]:
news.head()

,Category,Text
0,business,worldcom ex-boss launches defence lawyers defe...
1,business,german business confidence slides german busin...
2,business,bbc poll indicates economic gloom citizens in ...
3,tech,lifestyle governs mobile choice faster bett...
4,business,enron bosses in $168m payout eighteen former e...


In [0]:
import re
import nltk
nltk.download('stopwords')
#used once to download stopwords.
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
#cleaning and preprocessing

for i in range(0,len(news['Text'])):

  feed = re.sub('[^a-zA-Z]',' ',news['Text'][i])
  feed = feed.lower()
  feed = feed.split()
  ps = PorterStemmer()
  feed = [ps.stem(word) for word in feed ]#if not word in set(stopwords.words('english'))
  feed = ' '.join(feed)

  news['Text'][i]=feed



In [0]:
from sklearn.model_selection import train_test_split


In [0]:
train_df,test_df = train_test_split(news, test_size=0.2, random_state=45)
train_df.shape,test_df.shape

((1192, 2), (298, 2))

In [0]:
x_test = test_df['Text']
y_test = test_df[['Category']]

x_train = train_df['Text']
y_train = train_df[['Category']]

In [0]:
x_train.head()

959     portishead back after eight year cult british ...
234     ray charl studio becom museum a museum dedic t...
960     us firm pull out of iraq a us compani ha pull ...
123     debat need on donat cap a cap on donat to poli...
1480    qanta see profit fli to record australian airl...
Name: Text, dtype: object

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
#bag of words
parseur = CountVectorizer()
#create the document term matrix
XTrain = parseur.fit_transform(x_train)


#number of tokens
print(len(parseur.get_feature_names()))
#list of tokens
print(parseur.get_feature_names())

14584
['aa', 'aaa', 'aac', 'aadc', 'aaliyah', 'aamir', 'aaron', 'abacu', 'abandon', 'abat', 'abba', 'abbasi', 'abbott', 'abbrevi', 'abc', 'abd', 'abdomen', 'abdomin', 'abdullah', 'abdullatif', 'abeb', 'aberdeen', 'aberr', 'aberystwyth', 'abey', 'abeyi', 'abhorr', 'abi', 'abid', 'abigail', 'abil', 'abish', 'abl', 'abn', 'abolish', 'abort', 'abortionist', 'about', 'abov', 'abramovich', 'abroad', 'abrupt', 'absa', 'absenc', 'absent', 'absente', 'absolut', 'absorb', 'abstain', 'abstent', 'absurd', 'abu', 'abund', 'abus', 'abut', 'abuzz', 'abysm', 'ac', 'academ', 'academi', 'acapulco', 'acc', 'acceler', 'accent', 'accept', 'access', 'accid', 'accident', 'acclaim', 'accolad', 'accommod', 'accompani', 'accomplic', 'accomplish', 'accord', 'accordingli', 'accosi', 'account', 'accoust', 'accra', 'accredit', 'accret', 'accumul', 'accur', 'accuraci', 'accus', 'accustom', 'ace', 'aceh', 'achiev', 'achik', 'achil', 'acid', 'ackerman', 'acknowledg', 'acn', 'acoust', 'acpo', 'acquiesc', 'acquir', 'acq

In [0]:
#transform the sparse matrix into a numpy matrix
mdtTrain = XTrain.toarray()
#type of the matrix
print(type(mdtTrain))
#size of the matrix
print(mdtTrain.shape)
#frequency of the terms
freq_mots = np.sum(mdtTrain,axis=0)
print(freq_mots)
#arg sort
index = np.argsort(freq_mots)
print(index)
#print the terms and their frequency
imp = {'terme':np.asarray(parseur.get_feature_names())[index],'freq':freq_mots[index]}
print(pd.DataFrame(imp))

<class 'numpy.ndarray'>
(1192, 14584)
[1 4 3 ... 6 1 4]
[    0  4976  4974 ...  9055 13061 12910]
         terme   freq
0           aa      1
1        froth      1
2       froogl      1
3       ruptur      1
4      fronter      1
...        ...    ...
14579       in   9514
14580      and   9834
14581       of  10346
14582       to  13358
14583      the  28042

[14584 rows x 2 columns]


In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(mdtTrain)
print ("IDF:",tfidf.idf_)
tf_idf_matrix = tfidf.transform(mdtTrain)
print (tf_idf_matrix.todense())

IDF: [7.39107924 6.47478851 7.39107924 ... 6.13831627 7.39107924 6.98561413]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
#*** train the classifier
#import the class Logistic Regression
from sklearn.naive_bayes import GaussianNB
#instatiate the object
modelFirst = GaussianNB()
#perform the training process


tf_idf_matrix_nb=tf_idf_matrix.toarray()
modelFirst.fit(tf_idf_matrix_nb,y_train)
#size of coefficients matrix


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
#intercept
#print(modelFirst.intercept_)
#generate the document term matrix for the test set
#using the object learned from the train set
#create the document term matrix
mdtTest = parseur.transform(x_test)
#size of the matrix
print(mdtTest.shape)
#prediction for the test set


(298, 14584)


In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(mdtTest)
print ("IDF:",tfidf.idf_)
tf_idf_matrix_test = tfidf.transform(mdtTest)
print (tf_idf_matrix.todense())

IDF: [6.70044357 5.60183128 6.70044357 ... 6.00729639 6.70044357 6.70044357]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
tf_idf_matrix_test_nb=tf_idf_matrix_test.toarray()

In [0]:
predTest1 = modelFirst.predict(tf_idf_matrix_test_nb)

In [0]:
predTest1[1]

'entertainment'

In [0]:
tf_idf_matrix_test_nb

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [0]:
#import the metrics class for the performance measurement
from sklearn import metrics
#confusion matrix
mcTest = metrics.confusion_matrix(y_test,predTest1)
print(mcTest)
#recall
print(metrics.recall_score(y_test,predTest1,pos_label='spam',average='weighted'))
#precision
print(metrics.precision_score(y_test,predTest1,pos_label='spam',average='weighted'))
#accuracy rate
print(metrics.accuracy_score(y_test,predTest1))
#F1-Score
print(metrics.f1_score(y_test,predTest1,pos_label='spam',average='weighted'))

[[55  2  3  2  4]
 [ 2 51  2  3  2]
 [ 2  2 56  3  2]
 [ 1  0  1 57  0]
 [ 2  1  2  0 43]]
0.8791946308724832
0.8801184362865335
0.8791946308724832
0.8786925650927102


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'spam') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score

In [0]:
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss
from sklearn.metrics import classification_report

In [0]:
# define the parameter grid
n_estimators_list = list(range(50,201,50))
max_depth_list = [1,5,10,15,20,None]
rf_param_grid = dict(n_estimators=n_estimators_list,max_depth=max_depth_list)


# create the grid object
rf_model = RandomForestClassifier()
rf_grid_search_obj = GridSearchCV(rf_model, rf_param_grid, cv=5, scoring='accuracy', return_train_score=False)

# perform the grid search
#start = time.time()
rf_grid_search_obj.fit(tf_idf_matrix,y_train)
#end = time.time()
#print("total running time: %.2f seconds." %(end - start))


In [0]:
list(       zip(rf_grid_search_obj.cv_results_["params"],rf_grid_search_obj.cv_results_['mean_test_score'])         )

[({'max_depth': 1, 'n_estimators': 50}, 0.546946309904715),
 ({'max_depth': 1, 'n_estimators': 100}, 0.5663092015048697),
 ({'max_depth': 1, 'n_estimators': 150}, 0.5621321331880031),
 ({'max_depth': 1, 'n_estimators': 200}, 0.5461481663795225),
 ({'max_depth': 5, 'n_estimators': 50}, 0.8775078232129673),
 ({'max_depth': 5, 'n_estimators': 100}, 0.8892479167399177),
 ({'max_depth': 5, 'n_estimators': 150}, 0.8959565416124609),
 ({'max_depth': 5, 'n_estimators': 200}, 0.900977462114553),
 ({'max_depth': 10, 'n_estimators': 50}, 0.9244752294223127),
 ({'max_depth': 10, 'n_estimators': 100}, 0.9337083787489892),
 ({'max_depth': 10, 'n_estimators': 150}, 0.9227910411026334),
 ({'max_depth': 10, 'n_estimators': 200}, 0.9353820189163532),
 ({'max_depth': 15, 'n_estimators': 50}, 0.9395836995886221),
 ({'max_depth': 15, 'n_estimators': 100}, 0.9378889631166274),
 ({'max_depth': 15, 'n_estimators': 150}, 0.9488027847122114),
 ({'max_depth': 15, 'n_estimators': 200}, 0.9420941598396683),
 ({'ma

In [0]:
# define the parameter grid

penalty = ['l1', 'l2']
# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)
# Create hyperparameter options
grid = dict(C=C, penalty=penalty)

# create the grid object
lr_model = LogisticRegression()
lr_grid_search_obj_ = GridSearchCV(lr_model, grid, cv=5, scoring='accuracy', return_train_score=False)

# perform the grid search
#start = time.time()
lr_grid_search_obj_.fit(tf_idf_matrix,y_train)
#end = time.time()
#print("total running time: %.2f seconds." %(end - start))


In [0]:
list(       zip(lr_grid_search_obj_.cv_results_["params"],lr_grid_search_obj_.cv_results_['mean_test_score'])         )

[({'C': 1.0, 'penalty': 'l1'}, nan),
 ({'C': 1.0, 'penalty': 'l2'}, 0.9630744347948385),
 ({'C': 2.7825594022071245, 'penalty': 'l1'}, nan),
 ({'C': 2.7825594022071245, 'penalty': 'l2'}, 0.9672655673147921),
 ({'C': 7.742636826811269, 'penalty': 'l1'}, nan),
 ({'C': 7.742636826811269, 'penalty': 'l2'}, 0.9655884110966563),
 ({'C': 21.544346900318832, 'penalty': 'l1'}, nan),
 ({'C': 21.544346900318832, 'penalty': 'l2'}, 0.9689462395836996),
 ({'C': 59.94842503189409, 'penalty': 'l1'}, nan),
 ({'C': 59.94842503189409, 'penalty': 'l2'}, 0.9706198797510636),
 ({'C': 166.81005372000593, 'penalty': 'l1'}, nan),
 ({'C': 166.81005372000593, 'penalty': 'l2'}, 0.9714602158855173),
 ({'C': 464.15888336127773, 'penalty': 'l1'}, nan),
 ({'C': 464.15888336127773, 'penalty': 'l2'}, 0.9722970359691994),
 ({'C': 1291.5496650148827, 'penalty': 'l1'}, nan),
 ({'C': 1291.5496650148827, 'penalty': 'l2'}, 0.9731338560528814),
 ({'C': 3593.813663804626, 'penalty': 'l1'}, nan),
 ({'C': 3593.813663804626, 'pen